In [ ]:
using Clapeyron
model1 = PR(["water","methanol","hexane"],
userlocations = (; 
Mw = [18.02,32.04,86.18], 
Tc = [647.13,512.64,507.60], 
Pc = [219E+05,81.40E+05,30.40E05]), 
alpha_userlocations = (; acentricfactor = [0.3442,0.5620,0.3003]))

model2 = PCSAFT(["water","methanol","hexane"];userlocations=(;
Mw = [18.015,32.042,86.18], 
segment = [2.1945,1.5255,3.0576],
sigma = [2.229,3.23,3.7983],
epsilon = [141.66,188.9,236.77],
n_H=[2,1,0],
n_e=[2,1,0],
epsilon_assoc = Dict(
    (("methanol","e"),("methanol","H")) =>2899.5,
    (("methanol","H"),("water","e")) =>2351.835,
	(("methanol","e"),("water","H")) =>2351.835,
    (("water","H"),("water","e")) =>1804.17
    ),
bondvol = Dict(
    (("methanol","e"),("methanol","H")) =>0.035176,
    (("methanol","H"),("water","e")) =>0.080454702,
	(("methanol","e"),("water","H")) =>0.080454702,
    (("water","H"),("water","e")) =>0.2039
)
))


model3 = SAFTVRMie(["water","methanol","hexane"]; userlocations=(;
Mw = [18.015,32.042,86.18], 
segment = [1,1.407287576,2.1097],
sigma = [3.0063,3.458765399,4.423], 
epsilon = [ 266.68 299.9019139 290.8037543;
299.9019139 342.2672309 340.481724;
290.8037543 340.481724 354.38
], 
lambda_a = [6.,6.,6.],
lambda_r = [17.02,20.335876,17.203],
n_H=[2,1,0],
n_e=[2,1,0],
epsilon_assoc = Dict(
    (("methanol","e"),("methanol","H")) =>2254.51229075214,
    (("methanol","H"),("water","e")) =>2115.681616,
	(("methanol","e"),("water","H")) =>2115.681616,
    (("water","H"),("water","e")) =>1985.4
    ),
bondvol = Dict(
    (("methanol","e"),("methanol","H")) =>1.95E-28,  
    (("methanol","H"),("water","e")) => 1.43337E-28,
	(("methanol","e"),("water","H")) =>1.43337E-28,
    (("water","H"),("water","e")) =>1.0169E-28
)
))


In [ ]:
models=[model1,model2,model3]
N = 10000
T = 298.15
p = 1.013e5
x_LLEPR = zeros(N,6)
x_LLEPC = zeros(N,6)
x_LLEVR = zeros(N,6)
x_LLE=[x_LLEPR,x_LLEPC,x_LLEVR]



In [6]:
for k=1:3
z0 = [0.4999,0.0002,0.4999]
K0 = [0.001,10,1000]
idxend = N
(x,n,G) = tp_flash(models[k],p,T,z0,RRTPFlash(equilibrium=:lle,K0=K0))
x_LLE[k][1,1:3] = x[1,:]
x_LLE[k][1,4:6] = x[2,:]
K0 = x[1,:]./x[2,:]
z0[1] = x[1,1]/2+x[2,1]/2
z0[2] += 2/N
z0[3] = x[1,3]/2+x[2,3]/2
for i in 2:N
    (x,n,G) = tp_flash(models[k],p,T,z0,RRTPFlash(equilibrium=:lle,K0=K0))
    x_LLE[k][i,1:3] = x[1,:]
    x_LLE[k][i,4:6] = x[2,:]
    K0 = x[1,:]./x[2,:]
    z0 = (x_LLE[k][i,1:3]+x_LLE[k][i,4:6])-(x_LLE[k][i-1,1:3]+x_LLE[k][i-1,4:6])/2
    if abs(x[1,1]-x[2,1]) < 1e-3
        idxend=i-1
        break
    end
end
x_LLE[k] = vcat(x_LLE[k][1:idxend,1:3],reverse(x_LLE[k][1:idxend,4:6],dims=1));
end


In [ ]:
using PyCall
import PyPlot; const plt=PyPlot
ternary = pyimport("ternary")

using Conda
 Conda.add("python-ternary")
ternary = pyimport("ternary")
plt.clf()
figure, tax = ternary.figure(scale=1.0)
figure.set_size_inches(9, 9)

tax.boundary()
tax.gridlines(multiple=0.2, color="black")
tax.left_axis_label("hexane", fontsize=16, offset=0.15)
tax.right_axis_label("methanol", fontsize=16, offset=0.15)
tax.bottom_axis_label("water", fontsize=16, offset=0.02)

exp=[
    1	0	0
    0.9435	0.0565	0
    0.844	0.156	0
    0.6806	0.3194	0
    0.6352	0.3648	0
    0.5566	0.4434	0
    0.5276	0.4724	0
    0.4953	0.5038	9E-4
    0.4462	0.5521	0.0017
    0.4143	0.5825	0.0032
    0.3606	0.6337	0.0057
    0.321	0.671	0.008
    0.2873	0.7022	0.0105
    0.2578	0.7277	0.0145
    0.2337	0.7501	0.0162
    0.2056	0.7727	0.0217
    0.1717	0.8021	0.0262
    0.1455	0.8238	0.0307
    0.1205	0.8427	0.0368
    0.0936	0.8583	0.0481
    0.0509	0.8737	0.0754
    0.0177	0.868	0.1143
    0.0109	0.8341	0.155
    0	0.7677	0.2323
    0.0178	0	0.9822
    0.008	0.0141	0.9779
    0.0093	0.0097	0.981
    0.0107	0.0053	0.984
    0.0166	0.0141	0.9693
    0.0115	0.0118	0.9767
    0.013	0.0065	0.9805
    0.01	0.0073	0.9827
    0.0117	0.0116	0.9767
    0.009	0.0089	0.9821
    0.0108	0.008	0.9812
    0.0189	0.0227	0.9584
    0.0124	0.024	0.9636
    0.0096	0.0178	0.9726
    0.0065	0.0279	0.9656
    0.0162	0.0612	0.9226
    0.0074	0.0435	0.9491
    0.0155	0.0668	0.9177
    0.0127	0.0552	0.9321
    0.0073	0.0681	0.9246
    0.0124	0.0991	0.8885
    0.0069	0.1569	0.8362
    0.006	0.2115	0.7825
    0	0.2722	0.7278    
]


In [ ]:
tax.plot(x_LLE[1], linewidth=2.0,color="r",linestyle="-", label="PR")
tax.plot(x_LLE[2], linewidth=2.0,color="b",linestyle="--", label="PC-SAFT")
tax.plot(x_LLE[3], linewidth=2.0,color="g",linestyle="-.", label="SAFT-VR Mie")
tax.scatter(exp, color="black", label="")

tax.ticks(axis="lbr", multiple=0.2, linewidth=1,fontsize=12, tick_formats="%.1f", offset=0.02)

tax.get_axes().axis("off")
tax.clear_matplotlib_ticks()
tax.legend(loc="upper right",frameon=false,fontsize=14)
display(plt.gcf())
